In [0]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from string import punctuation
import re
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence,text
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Embedding,Flatten
from keras.models import load_model

stemmer = SnowballStemmer('english')
lemma = WordNetLemmatizer()

In [0]:
%%capture
import nltk
nltk.download("popular")

In [0]:
pd.options.display.max_columns=99
pd.options.display.max_colwidth=300
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [0]:
df = pd.read_csv(r'https://raw.githubusercontent.com/subhajitchoudhury/files/master/train.tsv',sep='\t')
testdf = pd.read_csv(r'https://raw.githubusercontent.com/subhajitchoudhury/files/master/test.tsv',sep='\t')            

In [0]:
df.head()

In [0]:
#df.loc[df.groupby('SentenceId')['PhraseId'].idxmin()]

In [0]:
#df.groupby('SentenceId')['PhraseId'].count().mean()

In [0]:
%%time
def cleantext(data):
    cleantextlist = []
    for text in data:
        text=re.sub('[^a-zA-Z]',' ',text)
        #text = " ".join([stemmer.stem(w) for w in word_tokenize(text.lower())])
        text = " ".join([lemma.lemmatize(w) for w in word_tokenize(text.lower())])
        cleantextlist.append(text)
    return cleantextlist
df['cleanPhrase'] = cleantext(df['Phrase'])
testdf['cleanPhrase'] = cleantext(testdf['Phrase'])

In [0]:
train_text = df.cleanPhrase.values
test_text = testdf.cleanPhrase.values
y = to_categorical(df.Sentiment)
num_classes = y.shape[1]
print("{},{},{},{}".format(train_text.shape,test_text.shape,y.shape,num_classes))

In [0]:
x_train,x_val,y_train,y_val = train_test_split(train_text,y,test_size=0.3,stratify=y)
print("{},{},{}".format(x_train.shape,x_val.shape,y_train.shape))

In [0]:
alltext = " ".join(x_train)
unique_words = len(set(word_tokenize(alltext)))
len_reviews = [len(word_tokenize(x)) for x in x_train]
maxlen = max(len_reviews)
print("{} : {}".format(unique_words,maxlen))

In [0]:
tokenizer = Tokenizer(num_words=unique_words)
tokenizer.fit_on_texts(x_train)
x_train_tok = tokenizer.texts_to_sequences(x_train)
x_val_tok = tokenizer.texts_to_sequences(x_val)
x_test_tok = tokenizer.texts_to_sequences(test_text)

In [0]:
x_train_pad = sequence.pad_sequences(x_train_tok,maxlen=maxlen)
x_val_pad = sequence.pad_sequences(x_val_tok,maxlen=maxlen)
x_test_pad = sequence.pad_sequences(x_test_tok,maxlen=maxlen)

In [0]:
# model1 = Sequential()
# model1.add(Embedding(unique_words,100,mask_zero=True))
# model1.add(LSTM(64,dropout=0.4,recurrent_dropout=0.4,return_sequences=True))
# model1.add(LSTM(32,dropout=0.4,recurrent_dropout=0.4,return_sequences=False))
# model1.add(Dense(num_classes,activation='softmax'))

In [0]:
# model1.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
# model1.summary()

In [0]:
%%time
# history1 = model1.fit(x_train_pad,y_train,validation_data=(x_val_pad,y_val),batch_size=128,epochs=5,verbose=1)
# model1.save('model3.h5')

In [0]:
#model2.save('model2.h5')

In [0]:
model1=load_model('model3.h5')

In [0]:
%%time
#y_pred1=model1.predict_classes(x_test_pad,verbose=1)
y_val_pred = model1.predict_classes(x_val_pad,verbose=1)

In [0]:
# sub=pd.read_csv(r'https://raw.githubusercontent.com/subhajitchoudhury/files/master/sampleSubmission.csv')
# sub.Sentiment=y_pred1
# sub.to_csv('submissionfile.csv',index=False)
# sub.head()

In [0]:
# sub['Phrase'] = testdf.Phrase
# print(sub.Sentiment.value_counts())
# print(df.Sentiment.value_counts())

In [0]:
reviewtext = ["this movie was bad.","this movie was good","the movie was the worst MOVIE I have seen","beautiful movie!"]
ctext = cleantext(reviewtext)
ttext = tokenizer.texts_to_sequences(ctext)
ptext = sequence.pad_sequences(ttext,maxlen=maxlen)
#print(ptext)
print(model1.predict_classes(ptext))

In [0]:
df[df.Sentiment==0][['cleanPhrase','Sentiment']][:10]

In [0]:
df2 = pd.DataFrame(df.loc[df.groupby('SentenceId')['PhraseId'].idxmin()][['cleanPhrase','Sentiment']])
df2[df2.Sentiment==4]

In [0]:
df1 = pd.DataFrame(y_val_pred, columns=['y_pred'])
df1['y_val'] = list(y_val)
df1['text'] = list(x_val)
df1['y_val1'] = [x.argmax() for x in df1.y_val]


In [0]:
df1[df1.y_pred!=df1.y_val1][['y_pred','y_val1','text',]]

In [0]:
from time import sleep
for i in range(1,1000):
  sleep(10)